In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
#于开源网站GitHub上找到女子比赛（性别差异）、场地不同的美网比赛的数据

In [ ]:
df_w = pd.read_excel('普适性测试_data.xlsx',sheet_name='Winbledon女单Final')
df_m = pd.read_excel('普适性测试_data.xlsx',sheet_name='美网男单初赛')

In [ ]:
df_w = df_w.fillna(method='backfill')
df_m = df_m.fillna(method='backfill')

In [ ]:
#整体步骤和对温网男子比赛数据进行分析是一样的

In [ ]:
def calculate_momentum_improved(df, index, window_size=3):
    start_index = max(index - window_size, 0)
    end_index = min(index + window_size + 1, len(df))
    time_window_df = df.iloc[start_index:end_index]

    p1_ovws = df.iloc[index]['p1_ovw_s']
    p2_ovws = df.iloc[index]['p2_ovw_s']
    p1_momentum = 0
    p2_momentum = 0
    
    p1_sets_won = time_window_df['p1_sets'].iloc[-1] - time_window_df['p1_sets'].iloc[0]
    p2_sets_won = time_window_df['p2_sets'].iloc[-1] - time_window_df['p2_sets'].iloc[0]
    p1_games_won = time_window_df['p1_games'].iloc[-1] - time_window_df['p1_games'].iloc[0]
    p2_games_won = time_window_df['p2_games'].iloc[-1] - time_window_df['p2_games'].iloc[0]

    serve_advantage_weight = 0.1
    p1_serve_advantage = (time_window_df[time_window_df['server'] == 1]['point_victor'] == 1).sum() * serve_advantage_weight
    p2_serve_advantage = (time_window_df[time_window_df['server'] == 2]['point_victor'] == 2).sum() * serve_advantage_weight
    
    game_win_weight = 0.2
    p1_game_win = (time_window_df['game_victor'] == 1).sum() * game_win_weight
    p2_game_win = (time_window_df['game_victor'] == 2).sum() * game_win_weight
    
    set_win_weight = 0.2
    p1_set_win = (time_window_df['set_victor'] == 1).sum() * set_win_weight
    p2_set_win = (time_window_df['set_victor'] == 2).sum() * set_win_weight
    
    p1_points_advantage = time_window_df['point_victor'].apply(lambda x: x == 1).sum() - time_window_df['point_victor'].apply(lambda x: x == 2).sum()
    p2_points_advantage = -p1_points_advantage
    p1_break_points_won = time_window_df['p1_break_pt_won'].sum()
    p2_break_points_won = time_window_df['p2_break_pt_won'].sum()
    p1_unforced_errors = -time_window_df['p1_unf_err'].sum()
    p2_unforced_errors = -time_window_df['p2_unf_err'].sum()
    p1_winners = time_window_df['p1_winner'].sum()
    p2_winners = time_window_df['p2_winner'].sum()
    p1_ace_advantage = time_window_df['p1_ace'].sum()
    p2_ace_advantage = time_window_df['p2_ace'].sum()
    p1_double_fault = time_window_df['p1_double_fault'].sum()
    p2_double_fault = time_window_df['p2_double_fault'].sum() 
    p1_net_pt = time_window_df['p1_net_pt'].sum()
    p2_net_pt = time_window_df['p2_net_pt'].sum()   
    
    p1_momentum = 0.0218*p1_serve_advantage + 0.0181*p1_points_advantage + 0.0705*p1_break_points_won + 0.0263*p1_winners + 0.0307*p1_game_win + 0.0804*p1_set_win + 0.0476*p1_ace_advantage + 0.0345*p1_net_pt + 0.0170*p1_sets_won + 0.0179*p1_games_won + 0.0777*p1_ovws - 0.0316*p1_unforced_errors - 0.0171*p1_double_fault
    p2_momentum = 0.0217*p2_serve_advantage + 0.0183*p2_points_advantage + 0.0767*p2_break_points_won + 0.0277*p2_winners + 0.0321*p2_game_win + 0.0842*p2_set_win + 0.0489*p2_ace_advantage + 0.0312*p2_net_pt + 0.0170*p2_sets_won + 0.0178*p2_games_won + 0.0863*p2_ovws - 0.0301*p2_unforced_errors - 0.0168*p2_double_fault
    return p1_momentum, p2_momentum

def cumsum_detection(series):
    diff_series = series.diff().fillna(0)
    cumsum_series = diff_series.cumsum()
    turning_points = []
    for i in range(1, len(cumsum_series)):
        if cumsum_series[i] * cumsum_series[i-1] <= 0:
            turning_points.append(i)
    return turning_points

def mark_indices_in_list(length, indices):
    # 初始化列表，长度为length，所有值为0
    marked_list = [0] * length
    
    # 在指定索引位置标记为1
    for index in indices:
        if index < length:
            marked_list[index] = 1
            
    return marked_list

In [ ]:
#温网女子比赛
momentum_values = [calculate_momentum_improved(df_w, index) for index in range(len(df_w))]

p1_momentum=[]
p2_momentum=[]
for i in momentum_values:
    p1_momentum.append(i[0])
    p2_momentum.append(i[1])

df_w['p1_momentum']=pd.DataFrame(p1_momentum)
df_w['p2_momentum']=pd.DataFrame(p2_momentum)

df_w['p1_momentum_diff'] = df_w['p1_momentum'].diff(1)
df_w.fillna(0,inplace=True)
df_w['p2_momentum_diff'] = df_w['p2_momentum'].diff(1)
df_w.fillna(0,inplace=True)

In [ ]:
#美网男子
momentum_values = [calculate_momentum_improved(df_m, index) for index in range(len(df_m))]

p1_momentum=[]
p2_momentum=[]
for i in momentum_values:
    p1_momentum.append(i[0])
    p2_momentum.append(i[1])

df_m['p1_momentum']=pd.DataFrame(p1_momentum)
df_m['p2_momentum']=pd.DataFrame(p2_momentum)

df_m['p1_momentum_diff'] = df_m['p1_momentum'].diff(1)
df_m.fillna(0,inplace=True)
df_m['p2_momentum_diff'] = df_m['p2_momentum'].diff(1)
df_m.fillna(0,inplace=True)

In [ ]:
from numpy import *
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual))

In [ ]:
#温网女子比赛的指标筛选
temp1_w=pd.get_dummies(df_w[['p1_score','p2_score','winner_shot_type','serve_width','serve_depth']])

temp2_w=df_w[['set_no', 'game_no','point_no', 
       'p1_sets', 'p2_sets', 'p1_games', 'p2_games','server', 'serve_no', 'point_victor', 'p1_points_won',
       'p2_points_won', 'game_victor', 'set_victor', 'p1_ace', 'p2_ace','p1_winner', 'p2_winner',
       'p1_double_fault', 'p2_double_fault', 'p1_unf_err', 'p2_unf_err', 'p1_net_pt', 'p2_net_pt',
       'p1_net_pt_won', 'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt',
       'p1_break_pt_won', 'p2_break_pt_won', 'p1_break_pt_missed','p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run','rally_count', 'speed_mph',
       'p1_ovw_s','p2_ovw_s','p1_momentum', 'p2_momentum']]

data_w=pd.concat([temp1_w,temp2_w],axis=1)
data_w.replace({True:1,False:0},inplace=True)
data_w.reset_index(inplace=True,drop=True)

In [ ]:
#美网男子比赛的指标筛选
temp1_m=pd.get_dummies(df_m[['p1_score','p2_score','winner_shot_type','serve_width','serve_depth']])

temp2_m=df_m[['set_no', 'game_no','point_no', 
       'p1_sets', 'p2_sets', 'p1_games', 'p2_games','server', 'serve_no', 'point_victor', 'p1_points_won',
       'p2_points_won', 'game_victor', 'set_victor', 'p1_ace', 'p2_ace','p1_winner', 'p2_winner',
       'p1_double_fault', 'p2_double_fault', 'p1_unf_err', 'p2_unf_err', 'p1_net_pt', 'p2_net_pt',
       'p1_net_pt_won', 'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt',
       'p1_break_pt_won', 'p2_break_pt_won', 'p1_break_pt_missed','p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run','rally_count', 'speed_mph',
       'p1_ovw_s','p2_ovw_s','p1_momentum', 'p2_momentum']]

data_m=pd.concat([temp1_m,temp2_m],axis=1)
data_m.replace({True:1,False:0},inplace=True)
data_m.reset_index(inplace=True,drop=True)

In [ ]:
dev_data = pd.read_csv('dev_data.csv')

In [ ]:
# 分离X和Y
X = dev_data[['p1_score_0', 'p1_score_1', 'p1_score_15', 'p1_score_2', 'p1_score_3',
       'p1_score_30', 'p1_score_4', 'p1_score_40', 'p1_score_5', 'p1_score_6',
       'p1_score_7', 'p1_score_8', 'p1_score_9', 'p1_score_AD', 'p2_score_0',
       'p2_score_1', 'p2_score_10', 'p2_score_15', 'p2_score_2', 'p2_score_3',
       'p2_score_30', 'p2_score_4', 'p2_score_40', 'p2_score_5', 'p2_score_6',
       'p2_score_7', 'p2_score_8', 'p2_score_9', 'p2_score_AD',
       'winner_shot_type_0', 'winner_shot_type_B', 'winner_shot_type_F',
       'serve_width_B', 'serve_width_BC', 'serve_width_BW', 'serve_width_C',
       'serve_width_W', 'serve_depth_CTL', 'serve_depth_NCTL', 'set_no',
       'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games',
       'server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won',
       'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner',
       'p2_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err',
       'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won',
       'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won',
       'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph',
       'p1_ovw_s', 'p2_ovw_s']]

Y1 = dev_data['p1_momentum_diff']

gbdt_model_1 = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
gbdt_model_1.fit(X, Y1)

In [ ]:
X_w = data_w[['p1_score_0', 'p1_score_15','p1_score_30', 'p1_score_40','p1_score_AD', 'p2_score_0',
       'p2_score_15','p2_score_30', 'p2_score_40','p2_score_AD',
       'winner_shot_type_0', 'winner_shot_type_B', 'winner_shot_type_F',
       'serve_width_B', 'serve_width_BC', 'serve_width_BW', 'serve_width_C',
       'serve_width_W', 'serve_depth_CTL', 'serve_depth_NCTL', 'set_no',
       'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games',
       'server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won',
       'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner',
       'p2_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err',
       'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won',
       'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won',
       'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph',
       'p1_ovw_s', 'p2_ovw_s']]

X_m = data_m[['p1_score_0', 'p1_score_15','p1_score_30','p1_score_40','p1_score_AD', 'p2_score_0',
       'p2_score_15','p2_score_30', 'p2_score_40','p2_score_AD',
       'winner_shot_type_0', 'winner_shot_type_B', 'winner_shot_type_F',
       'serve_width_B', 'serve_width_BC', 'serve_width_BW', 'serve_width_C',
       'serve_width_W', 'serve_depth_CTL', 'serve_depth_NCTL', 'set_no',
       'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games',
       'server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won',
       'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner',
       'p2_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err',
       'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won',
       'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won',
       'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph',
       'p1_ovw_s', 'p2_ovw_s']]

In [ ]:
#合并一下
X_aw = pd.DataFrame(columns=['p1_score_1', 'p1_score_2', 'p1_score_3', 'p1_score_4', 'p1_score_5', 'p1_score_6','p1_score_7', 'p1_score_8', 'p1_score_9',
       'p2_score_1', 'p2_score_10', 'p2_score_2', 'p2_score_3','p2_score_4','p2_score_5', 'p2_score_6',
       'p2_score_7', 'p2_score_8', 'p2_score_9'])
X_pw = pd.concat([X_w,X_aw],axis=1)
X_pw.fillna(0,inplace=True)
X_pw = X_pw[['p1_score_0', 'p1_score_1', 'p1_score_15', 'p1_score_2', 'p1_score_3',
       'p1_score_30', 'p1_score_4', 'p1_score_40', 'p1_score_5', 'p1_score_6',
       'p1_score_7', 'p1_score_8', 'p1_score_9', 'p1_score_AD', 'p2_score_0',
       'p2_score_1', 'p2_score_10', 'p2_score_15', 'p2_score_2', 'p2_score_3',
       'p2_score_30', 'p2_score_4', 'p2_score_40', 'p2_score_5', 'p2_score_6',
       'p2_score_7', 'p2_score_8', 'p2_score_9', 'p2_score_AD',
       'winner_shot_type_0', 'winner_shot_type_B', 'winner_shot_type_F',
       'serve_width_B', 'serve_width_BC', 'serve_width_BW', 'serve_width_C',
       'serve_width_W', 'serve_depth_CTL', 'serve_depth_NCTL', 'set_no',
       'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games',
       'server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won',
       'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner',
       'p2_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err',
       'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won',
       'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won',
       'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph',
       'p1_ovw_s', 'p2_ovw_s']]
#70多个要素

In [ ]:
#运用GBDT模型进行预测
Y_pred_w = gbdt_model_1.predict(X_pw)
df_w['p1_momentum_diff_pred'] = Y_pred_w

In [ ]:
print("平均绝对误差:",mean_absolute_error(df_w['p1_momentum_diff'], Y_pred_w))
print("r2_score",r2_score(df_w['p1_momentum_diff'],Y_pred_w))

In [ ]:
def cumsum_detection_diff(series):
    cumsum_series = series.cumsum()
    turning_points = []
    for i in range(1, len(cumsum_series)):

        if cumsum_series[i] * cumsum_series[i-1] <= 0:
            turning_points.append(i)
    
    return turning_points

In [ ]:
#绘制温网女子比赛转折情况
elapsed_seconds = df_w['elapsed_time']

p1_turning_points = cumsum_detection_diff(df_w['p1_momentum_diff'])
pred_turning_points = cumsum_detection_diff(df_w['p1_momentum_diff_pred'])

plt.figure(figsize=(14, 6))
plt.plot(elapsed_seconds, df_w['p1_momentum'], label=df_w['player1'].iloc[0] + " Momentum", color='royalblue')

plt.scatter(elapsed_seconds.iloc[p1_turning_points], df_w['p1_momentum'].iloc[p1_turning_points], color='darkviolet', label='P1 Turning Points', zorder=5,alpha=0.5)
plt.scatter(elapsed_seconds.iloc[pred_turning_points], df_w['p1_momentum'].iloc[pred_turning_points], color='darkorange', label='Predictive Turning Points', zorder=5,alpha=0.5)

plt.legend()
plt.title('Momentum Change and Turning Points')
plt.xlabel('Elapsed Time (seconds)')
plt.ylabel('P1 Momentum')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
X_am = pd.DataFrame(columns=['p1_score_1', 'p1_score_2', 'p1_score_3', 'p1_score_4', 'p1_score_5', 'p1_score_6','p1_score_7', 'p1_score_8', 'p1_score_9',
       'p2_score_1', 'p2_score_10', 'p2_score_2', 'p2_score_3','p2_score_4','p2_score_5', 'p2_score_6',
       'p2_score_7', 'p2_score_8', 'p2_score_9'])
X_pm = pd.concat([X_m,X_am],axis=1)
X_pm.fillna(0,inplace=True)
X_pm = X_pm[['p1_score_0', 'p1_score_1', 'p1_score_15', 'p1_score_2', 'p1_score_3',
       'p1_score_30', 'p1_score_4', 'p1_score_40', 'p1_score_5', 'p1_score_6',
       'p1_score_7', 'p1_score_8', 'p1_score_9', 'p1_score_AD', 'p2_score_0',
       'p2_score_1', 'p2_score_10', 'p2_score_15', 'p2_score_2', 'p2_score_3',
       'p2_score_30', 'p2_score_4', 'p2_score_40', 'p2_score_5', 'p2_score_6',
       'p2_score_7', 'p2_score_8', 'p2_score_9', 'p2_score_AD',
       'winner_shot_type_0', 'winner_shot_type_B', 'winner_shot_type_F',
       'serve_width_B', 'serve_width_BC', 'serve_width_BW', 'serve_width_C',
       'serve_width_W', 'serve_depth_CTL', 'serve_depth_NCTL', 'set_no',
       'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games',
       'server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won',
       'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner',
       'p2_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err',
       'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won',
       'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won',
       'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed',
       'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph',
       'p1_ovw_s', 'p2_ovw_s']]

In [ ]:
Y_pred_m = gbdt_model_1.predict(X_pm)
df_m['p1_momentum_diff_pred'] = Y_pred_m
print("平均绝对误差:",mean_absolute_error(df_m['p1_momentum_diff'], Y_pred_m))
print("r2_score",r2_score(df_m['p1_momentum_diff'],Y_pred_m))

In [ ]:
#绘制美网男子比赛转折情况
elapsed_seconds = df_m['elapsed_time']

p1_turning_points = cumsum_detection_diff(df_m['p1_momentum_diff'])
pred_turning_points = cumsum_detection(df_m['p1_momentum_diff_pred'])

plt.figure(figsize=(14, 6))
plt.plot(elapsed_seconds, df_m['p1_momentum'], label=df_m['player1'].iloc[0] + " Momentum", color='royalblue')

plt.scatter(elapsed_seconds.iloc[p1_turning_points], df_m['p1_momentum'].iloc[p1_turning_points], color='darkviolet', label='P1 Turning Points', zorder=5,alpha=0.5)
plt.scatter(elapsed_seconds.iloc[pred_turning_points], df_m['p1_momentum'].iloc[pred_turning_points], color='darkorange', label='Predictive Turning Points', zorder=5,alpha=0.5)

plt.legend()
plt.title('Momentum Change and Turning Points')
plt.xlabel('Elapsed Time (seconds)')
plt.ylabel('P1 Momentum')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#GBDT模型，用于其他性别、其他网球场地的比赛，
#也有较好的预测能力，同时由可视化的结果可知，对势头转折点的识别合理。有普适性。